In [7]:
import re
import torch
from langchain.document_loaders import PyPDFLoader
from langchain_huggingface import HuggingFaceEmbeddings
# from langchain_experimental.text_splitter import SemanticChunker

In [8]:
import os
# 캐시 디렉토리 설정
cache_dir = '/Volumes/MINDB/24년/SW아카데미/LangEyE/App/src/weights'
os.makedirs(cache_dir, exist_ok=True)
os.environ['TRANSFORMERS_CACHE'] = cache_dir
os.environ['HF_HOME'] = cache_dir
os.environ['HF_DATASETS_CACHE'] = os.path.join(cache_dir, 'datasets')
os.environ['HUGGINGFACE_HUB_CACHE'] = cache_dir
os.environ['TORCH_HOME'] = os.path.join(cache_dir, 'torch')

In [353]:
import re
from pypdf import PdfReader

pdf_path = '/Volumes/MINDB/24년/SW아카데미/LangEyE/crawling/장애인복지법.pdf'
reader = PdfReader(pdf_path)
sample_text = ''
for page in reader.pages:
    
    temp = page.extract_text()
    temp = temp[temp.find('장애인복지법')+6:]
    sample_text += temp



In [354]:
sample_text

'\n \n장애인복지법\n[시행 2024. 10. 22.] [법률 제20511호, 2024. 10. 22., 타법개정]\n보건복지부 (장애인정책과) 044-202-3282\n보건복지부 (장애인권익지원과) 044-202-3301, 3309\n       제1장 총칙\n \n제1조(목적) 이 법은 장애인의 인간다운 삶과 권리보장을 위한 국가와 지방자치단체 등의 책임을 명백히 하고, 장애발\n생 예방과 장애인의 의료ㆍ교육ㆍ직업재활ㆍ생활환경개선 등에 관한 사업을 정하여 장애인복지대책을 종합적으로\n추진하며, 장애인의 자립생활ㆍ보호 및 수당지급 등에 관하여 필요한 사항을 정하여 장애인의 생활안정에 기여하는\n등 장애인의 복지와 사회활동 참여증진을 통하여 사회통합에 이바지함을 목적으로 한다.\n \n제2조(장애인의 정의 등) ①“장애인”이란 신체적ㆍ정신적 장애로 오랫동안 일상생활이나 사회생활에서 상당한 제약을\n받는 자를 말한다.\n②이 법을 적용받는 장애인은 제1항에 따른 장애인 중 다음 각 호의 어느 하나에 해당하는 장애가 있는 자로서 대\n통령령으로 정하는 장애의 종류 및 기준에 해당하는 자를 말한다.\n1. “신체적 장애”란 주요 외부 신체 기능의 장애, 내부기관의 장애 등을 말한다.\n2. “정신적 장애”란 발달장애 또는 정신 질환으로 발생하는 장애를 말한다.\n③ “장애인학대”란 장애인에 대하여 신체적ㆍ정신적ㆍ정서적ㆍ언어적ㆍ성적 폭력이나 가혹행위, 경제적 착취, 유기\n또는 방임을 하는 것을 말한다.<신설 2012. 10. 22., 2015. 6. 22.>\n④ “장애인학대관련범죄”란 장애인학대로서 다음 각 호의 어느 하나에 해당하는 죄를 말한다.<신설 2020. 12. 29.>\n1. 「형법」 제2편제24장 살인의 죄 중 제250조(살인, 존속살해), 제252조(촉탁, 승낙에 의한 살인 등), 제253조(위계\n등에 의한 촉탁살인 등) 및 제254조(미수범)의 죄\n2. 「형법」 제2편제25장 상해와 폭행의 죄 중 제257조(상해, 존속상해), 제258조(중

In [343]:
import re

def extract_law_title(text):
    """제목 추출"""
    title_pattern = r'\n(.+?)\n'
    title_match = re.search(title_pattern, text)
    if title_match:
        return title_match.group(1).strip()
    else:
        assert False, "제목을 찾을 수 없습니다."

def is_valid(text):
    """[시행 2024. 10. 22.]"""
    valid_pattern = r'\[시행\s(\d{4}\.\s\d{2}\.\s\d{2}\.)\]'
    match_pattern = re.search(valid_pattern, text)
    
    ## 날짜 형식으로 변경
    times = match_pattern.group(1).replace('.', '-').replace(' ', '')[:-1]

    if match_pattern:
        return True, times
    
def load_pdf_text(pdf_path):
    reader = PdfReader(pdf_path)
    raw_text = ""
    for page in reader.pages:
        raw_text += page.extract_text()
    return raw_text

def extract_article_number(text):
    """조문 번호 추출 (예: '제2조(장애인의 정의 등)' -> '2')"""
    article_pattern = r'(제\d+조)(?:의\d+)?(?:\([^)]+\))?\s*\([^)]+\)|(제\d+조)(?:의\d+)?\s*\([^)]+\)'
    article_match = re.search(article_pattern, text)
    if article_match:
        return article_match.group(1)
    return None

def extract_paragraph_number(text):
    """항 번호 추출 (①, ②, 1., 2. 등)"""
    # 원문자 항 번호 찾기
    circle_num = re.search(r'[①②③④⑤⑥⑦⑧⑨⑩]', text)
    if circle_num:
        circle_to_num = {'①':'1', '②':'2', '③':'3', '④':'4', '⑤':'5',
                        '⑥':'6', '⑦':'7', '⑧':'8', '⑨':'9', '⑩':'10'}
        return circle_to_num.get(circle_num.group())
    
    # 숫자 항 번호 찾기 (1., 2. 등)
    num_match = re.search(r'^(\d+)\.', text.strip())
    if num_match:
        return num_match.group(1)
    return None

def extract_subject(text):
    """조문의 주제 추출 (예: '제2조(장애인의 정의 등)' -> '장애인의 정의 등')"""
    article_pattern = r'제\d+조(?:의\d+)?(?:\([^)]+\))?\s*\([^)]+\)|제\d+조(?:의\d+)?\s*\([^)]+\)'
    article_match = re.search(article_pattern, text)

    subject_pattern = r'\(([^)]+)\)'
    subject_match = re.search(subject_pattern, article_match.group(0))

    if subject_match:
        return subject_match.group(1)
    

pdf_path = "/Volumes/MINDB/24년/SW아카데미/LangEyE/crawling/장애인복지법.pdf"
sample_text = load_pdf_text(pdf_path)
extract_law_title(sample_text), \
    is_valid(sample_text), \
    extract_article_number(sample_text), \
    extract_paragraph_number(sample_text), \
    extract_subject(sample_text)

('장애인복지법', (True, '2024-10-22'), '제1조', '1', '목적')

In [3]:
from langchain.docstore.document import Document
import re
from pypdf import PdfReader

class LegalText:
    def __init__(self, pdf_path):
        self.docs = self.load_pdf_text(pdf_path)
        self.documents = []
        self.law_name = self.extract_law_title(self.docs)
        self.valid , self.valid_date = self.is_valid(self.docs)
        self.pre_doc = self.set_doc(self.docs)

    def load_pdf_text(self, pdf_path):
        reader = PdfReader(pdf_path)
        docs = ''
        for page in reader.pages:
            temp = page.extract_text()
            temp = temp[temp.find('장애인복지법')+6:]
            docs += temp
        return docs

    def is_valid(self, text):
        """[시행 2024. 10. 22.]"""
        valid_pattern = r'\[시행\s(\d{4}\.\s\d{2}\.\s\d{2}\.)\]'
        match_pattern = re.search(valid_pattern, text)
        
        ## 날짜 형식으로 변경
        times = match_pattern.group(1).replace('.', '-').replace(' ', '')[:-1]

        if match_pattern:
            return True, times

    def extract_law_title(self, text):
        """제목 추출"""
        title_pattern = r'\n(.+?)\n'
        title_match = re.search(title_pattern, text)
        if title_match:
            return title_match.group(1).strip()
        else:
            assert False, "제목을 찾을 수 없습니다."

    ##### 초기 설정 ######
    
    def extract_main_articles(self, text):
        """주요 조문 추출"""
        pattern = r'(제\d+조(?:의\d+)?\s*\([^)]+\))\s*(.*?)(?=\n\s*\n제\d+조(?:의\d+)?\s*\([^)]+\)|$)'
        match = re.findall(pattern, text, re.DOTALL)
        # print(match[0])
        return match

    def set_doc(self, text):
        # 장(章) 제목 패턴
        pattern = r'제\d+장\s+([^\n]+)\n\s\n'
        chapter_list = re.split(pattern, text)
        paragraph_title = []
        number = 1
        
        for i in range(len(chapter_list)):
            ## 첫 장 패스
            if i == 0:
                continue

            ## 장 제목 : 홀수
            if i % 2 != 0:
                paragraph_title.append(f'제{number}장 {chapter_list[i]}')
                number += 1
                continue
            ## 조문 : 짝수
            else:
                # print('=======')
                # print(chapter_list[i])
                # print(self.extract_main_articles(chapter_list[i]))
                
                paragraph = paragraph_title.pop(0)

                for j in range(len(self.extract_main_articles(chapter_list[i]))):
                    self.documents.append(Document(
                        page_content=self.extract_main_articles(chapter_list[i])[j][1],
                        metadata={
                            "law_title" : self.law_name, ## 법 제목
                            "effective_date" : self.valid_date, ## 시행 일자
                            "paragraph" : paragraph, ## 항 제목
                            "article_number" : self.extract_main_articles(chapter_list[i])[j][0], ## 조문 번호
                            "document_type" : "법률",
                            "is_valid" : self.valid,
                            "legal_area" : "장애인복지",
                        }
                    ))

        return self.documents
pdf_path = '/Volumes/MINDB/24년/SW아카데미/LangEyE/crawling/장애인복지법.pdf'
LegalText(pdf_path).documents

[Document(metadata={'law_title': '', 'effective_date': '2024-10-22', 'paragraph': '제1장 총칙', 'article_number': '제1조(목적)', 'document_type': '법률', 'is_valid': True, 'legal_area': '장애인복지'}, page_content='이 법은 장애인의 인간다운 삶과 권리보장을 위한 국가와 지방자치단체 등의 책임을 명백히 하고, 장애발\n생 예방과 장애인의 의료ㆍ교육ㆍ직업재활ㆍ생활환경개선 등에 관한 사업을 정하여 장애인복지대책을 종합적으로\n추진하며, 장애인의 자립생활ㆍ보호 및 수당지급 등에 관하여 필요한 사항을 정하여 장애인의 생활안정에 기여하는\n등 장애인의 복지와 사회활동 참여증진을 통하여 사회통합에 이바지함을 목적으로 한다.'),
 Document(metadata={'law_title': '', 'effective_date': '2024-10-22', 'paragraph': '제1장 총칙', 'article_number': '제2조(장애인의 정의 등)', 'document_type': '법률', 'is_valid': True, 'legal_area': '장애인복지'}, page_content='①“장애인”이란 신체적ㆍ정신적 장애로 오랫동안 일상생활이나 사회생활에서 상당한 제약을\n받는 자를 말한다.\n②이 법을 적용받는 장애인은 제1항에 따른 장애인 중 다음 각 호의 어느 하나에 해당하는 장애가 있는 자로서 대\n통령령으로 정하는 장애의 종류 및 기준에 해당하는 자를 말한다.\n1. “신체적 장애”란 주요 외부 신체 기능의 장애, 내부기관의 장애 등을 말한다.\n2. “정신적 장애”란 발달장애 또는 정신 질환으로 발생하는 장애를 말한다.\n③ “장애인학대”란 장애인에 대하여 신체적ㆍ정신적ㆍ정서적ㆍ언어적ㆍ성적 폭력이나 가혹행위, 경제적 착취, 유기\n또는 방임을 하는 것을 말한다.<신설 2012. 10. 22., 2015. 6. 22.

In [ ]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain_huggingface import HuggingFacePipeline
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_experimental.text_splitter import SemanticChunker

# 캐시 디렉토리 설정
cache_dir = './weights'
os.makedirs(cache_dir, exist_ok=True)
os.environ['TRANSFORMERS_CACHE'] = cache_dir
os.environ['HF_HOME'] = cache_dir
os.environ['HF_DATASETS_CACHE'] = os.path.join(cache_dir, 'datasets')
os.environ['HUGGINGFACE_HUB_CACHE'] = cache_dir
os.environ['TORCH_HOME'] = os.path.join(cache_dir, 'torch')

def load_rag_model():
    docs = LegalText(pdf_path).documents

    # 임베딩 모델 설정
    if torch.backends.mps.is_available():
        device = 'mps'
    elif torch.cuda.is_available():
        device = 'cuda'
    else:
        device = 'cpu'
    
    model_name = "upskyy/bge-m3-korean"
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs={'device': device},
        encode_kwargs={'normalize_embeddings': True},
        cache_folder=cache_dir
    )

    # 임베딩 기반 문서 분할
    text_splitter = SemanticChunker(embeddings)
    splits = text_splitter.split_documents(docs)
    
    # 벡터스토어
    vectorstore = FAISS.from_documents(
        documents=splits,
        embedding=embeddings
    )
    retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

    # 프롬프트 템플릿 (사용자 정보 포함)
    template = """
    당신은 시각장애인을 대상으로 복지 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다.

    사용자 정보:
    {user_info}

    질문: {question}

    관련 문서:
    {context}

    위 정보를 바탕으로 사용자의 상황에 맞는 복지 정보를 [관련 법안 혹은 근거], [대상], [혜택 내용], [신청 방법], [문의처]를 포함하여 안내해주세요.
    특히 사용자의 장애등급, 소득분위 등을 고려하여 맞춤형 정보를 제공해주세요.
    """
    
    prompt = PromptTemplate.from_template(template)

    # 모델 로드
    tokenizer = AutoTokenizer.from_pretrained(
        "davidkim205/Ko-Llama-3-8B-Instruct",
        cache_dir=cache_dir
    )
    model = AutoModelForCausalLM.from_pretrained(
        "davidkim205/Ko-Llama-3-8B-Instruct",
        device_map="auto",
        torch_dtype=torch.float16,
        cache_dir=cache_dir
    )
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.7,
        top_p=0.95,
        repetition_penalty=1.15
    )
    llm = HuggingFacePipeline(pipeline=pipe)

    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)

    # RAG 체인 생성 (사용자 정보 포함)
    def create_chain_with_user_info(user_info: str):
        rag_chain = (
            {
                "context": retriever | format_docs,
                "question": RunnablePassthrough(),
                "user_info": lambda _: user_info
            }
            | prompt
            | llm
            | StrOutputParser()
        )
        return rag_chain

    return create_chain_with_user_info

In [ ]:
def preprocess_legal_text(self):
    """법률 텍스트 전처리"""
    chunks = []
    law_title = self.extract_law_title(self.docs)
    
    # 조문 단위로 먼저 분리
    article_pattern = r'(제\d+조(?:\의\d+)?)'
    articles = re.split(article_pattern, self.docs)
    
    # articles[0]은 법률 제목 부분이므로 건너뜀
    for i in range(1, len(articles), 2):
        if i + 1 >= len(articles):  # 인덱스 범위 체크
            break
            
        article_num = articles[i]  # 조문 번호 (예: 제32조)
        article_content = articles[i+1]  # 해당 조문의 내용
        
        # 빈 내용이면 건너뜀
        if not article_content.strip():
            continue
            
        # 항 단위로 분리
        paragraph_pattern = r'([①②③④⑤⑥⑦⑧⑨]|[0-9]+\.)'
        paragraphs = re.split(paragraph_pattern, article_content)
        
        # 항이 없는 경우 전체를 하나의 청크로
        if len(paragraphs) == 1:
            processed_chunk = {
                "law_name": law_title,
                "article_number": self.extract_article_number(article_num),
                "article_text": article_content.strip(),
                "paragraph_number": None,
                "content": article_content.strip(),
                "article_title": self.extract_subject(article_content)
            }
            chunks.append(processed_chunk)
            continue
        
        # 항이 있는 경우 각 항을 별도 청크로
        for j in range(1, len(paragraphs), 2):
            if j + 1 >= len(paragraphs):  # 인덱스 범위 체크
                break
                
            paragraph_num = paragraphs[j]
            paragraph_content = paragraphs[j+1]
            
            if not paragraph_content.strip():  # 빈 내용이면 건너뜀
                continue
            
            processed_chunk = {
                "law_name": law_title,
                "article_number": self.extract_article_number(article_num),
                "article_text": article_content.strip(),
                "paragraph_number": self.extract_paragraph_number(paragraph_num),
                "content": paragraph_content.strip(),
                "article_title": self.extract_subject(article_content)
            }
            chunks.append(processed_chunk)
    
    return chunks

def create_documents(self):
    """청크를 Langchain Document 객체로 변환"""
    documents = []
    for chunk in self.chunks:
        doc = Document(
            page_content=chunk["content"],
            metadata={
                "law_name": chunk["law_name"],
                "article_number": chunk["article_number"],
                "article_text": chunk["article_text"],
                "paragraph_number": chunk["paragraph_number"],
                "article_title": chunk["article_title"]
            }
        )
        documents.append(doc)
    return documents

In [ ]:
import re
from typing import Dict, List, Optional

class LegalDocumentParser:
    def __init__(self):
        self.current_law = ""
        self.current_chapter = ""
        self.current_section = ""
        
    def parse_document(self, text: str) -> List[Dict]:
        documents = []
        
        # 법률 제목 파싱
        law_title_match = re.search(r'(「.*?」)', text)
        if law_title_match:
            self.current_law = law_title_match.group(1)
            
        # 조문 단위로 분리
        articles = re.split(r'(제\d+조)', text)
        
        for i in range(1, len(articles), 2):
            if i+1 < len(articles):
                article_num = articles[i]
                content = articles[i+1]
                
                # 항 분리
                paragraphs = re.split(r'(①|②|③|④|⑤|⑥|⑦|⑧|⑨|⑩)', content)
                
                for j in range(1, len(paragraphs), 2):
                    if j+1 < len(paragraphs):
                        paragraph_num = paragraphs[j]
                        paragraph_content = paragraphs[j+1]
                        
                        metadata = {
                            "law_title": self.current_law,                # 법률 제목
                            "article_number": article_num.strip(),        # 조번호
                            "paragraph_number": paragraph_num,            # 항번호
                            "document_type": "법률",                      # 문서 유형
                            "effective_date": "2024. 10. 22.",           # 시행일자
                            "amendment_number": "제20511호",              # 개정 번호
                            "section_title": self.current_section,        # 장/절 제목
                            "chapter_title": self.current_chapter,        # 장 제목
                            "is_valid": True,                            # 현행 여부
                            "legal_area": "장애인복지",                    # 법률 분야
                            "content_type": "본문",                       # 내용 유형 (본문/부칙/별표 등)
                            "related_articles": self._find_related_articles(paragraph_content)  # 관련 조항
                        }
                        
                        documents.append({
                            "content": paragraph_content.strip(),
                            "metadata": metadata
                        })
        
        return documents
    
    def _find_related_articles(self, text: str) -> List[str]:
        """본문에서 참조하는 다른 조항들을 찾아내는 메서드"""
        related = re.findall(r'제\d+조', text)
        return list(set(related))

# 사용 예시
parser = LegalDocumentParser()
documents = parser.parse_document(legal_text)